<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Earth Analytics Education - Climate 101 Workshop

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4
import xarray as xr
import rioxarray
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import geopandas as gpd
import earthpy as et

# Plotting options
sns.set(font_scale=1.3)
sns.set_style("white")

## Challenge 1 - Select Your Data

In [2]:
# Model options between 0-19
model = 2
# Options 0-8 will work for var. Var maps to the variable name below
var = 5
# Options range from 0-2
scenario = 1

In [3]:
dir_path = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'

# These are the variable options for the met data
variable_name = ('tasmax', 'tasmin', 'rhsmax',
                 'rhsmin', 'pr', 'rsds',
                 'uas', 'vas', 'huss')

# These are var options in long form
var_long_name = ('air_temperature', 
                 'air_temperature', 
                 'relative_humidity',
                 'relative_humidity',
                 'precipitation', 
                 'surface_downwelling_shortwave_flux_in_air',
                 'eastward_wind', 
                 'northward_wind', 
                 'specific_humidity')

# Models to chose from
model_name = ('bcc-csm1-1', 
              'bcc-csm1-1-m', 
              'BNU-ESM', 
              'CanESM2',
              'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0',
              'GFDL-ESM2G', 'GFDL-ESM2M', 'HadGEM2-CC365',
              'HadGEM2-ES365',
              'inmcm4', 'IPSL-CM5A-MR', 'IPSL-CM5A-LR',
              'IPSL-CM5B-LR',
              'MIROC5', 'MIROC-ESM', 'MIROC-ESM-CHEM',
              'MRI-CGCM3', 'NorESM1-M')

# Scenarios
scenario_type = ('historical', 'rcp45', 'rcp85')

# Year start and ends (historical vs projected)
year_start = ('1950', '2006', '2006')
year_end = ('2005', '2099', '2099')
run_num = [1] * 20
run_num[4] = 6  # setting CCSM4 with run 6
domain = 'CONUS'

In [4]:
time = year_start[scenario]+'_' + year_end[scenario]
print("Your selected time period is:", time)

Your selected time period is: 2006_2099


In [5]:
# This is only going to provide monthly data
file_name = ('agg_macav2metdata_' +
             str(variable_name[var]) +
             '_' +
             str(model_name[model]) +
             '_r' +
             str(RUN_NUM[model])+'i1p1_' +
             str(scenario_type[scenario]) +
             '_' +
             time + '_' +
             domain + '_monthly.nc')

print("You are accessing:\n", file_name, "\n data in netcdf format")

NameError: name 'RUN_NUM' is not defined

In [ ]:
full_file_path = dir_path + file_name
full_file_path

## Challenge 2 - Run the Cell Below to Open Your Data

Run the cell below to open your dataset.


In [ ]:
# Open the data
with xr.open_dataset(full_file_path) as file_nc:
    max_temp_xr = file_nc

# View xarray object
max_temp_xr

## Challenge 3 - Subset Your Data

Add text here that tells them to select a lat / lon and start and end time below...
you can populate the cell below with a starting set of values. 

Ok nathan - please add the following to this notebook. take a workflow from the intro to netcdf notebook
that opens the data / slices it and calculates a max value for each time step.

Then plot the data (challenge 4 below)

Add code to slice the data by both TIME and lat / lon

Nathan here you will add all of the code needed to produce the plots


In [ ]:
# NATHAN - populate these with example lat / long and times that will make the notebook run.
# The challenge will then  be for them to adjust the code in this cell and then the notebook will
# Plot a time series of max precip for each time step in the range they choose.
latitude =1
longitude =1
time_start =1
time_end = 1

In [ ]:
# Nathan add a few cells that calculate a max value for each time step and then a cell that plots the data

# Maybe on the plot cell you can tell them to change the title??

## Challenge 4 - Modify your plot

Add code below to plot the data as a time series. Then give them instructions to modify the colors, title, etc

## Challenge 5  - Export your data to a csv file

In the cell below, export the data that you processed above to a dataframe and then a .csv file. 
You can open up and view the `.csv` file using the jupyterHub

In [ ]:
# Your code here